In [ ]:
import pysm3
import pysm3.units as u
import numpy as np
import numpy.ma as ma
import healpy as hp
import pickle
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pickle
from astropy.io import fits
import pandas as pd
from importlib import reload

from qubic import camb_interface as qc
import fgbuster as fgb

# Specific qubic modules
from qubicpack.utilities import Qubic_DataDir
from pysimulators import FitsArray
#from mpi4py import MPI
# from pyoperators import MPI
import pysm3 as pysm
from pysm3 import utils

import qubic
from qubic import QubicSkySim as qss
from qubic import fibtools as ft
from qubic import camb_interface as qc
from qubic import SpectroImLib as si
from qubic import NamasterLib as nam
from qubic import mcmc

#### This is a temporary library
import compsep_jc as csjc


import warnings
warnings.filterwarnings("ignore")

import qubic
from qubic import NamasterLib as nam

center = qubic.equ2gal(0, -57)

rc('figure', figsize=(16, 10))
rc('font', size=15)

#### General Stuff ####
stk = ['I', 'Q', 'U']


### Initialize
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])

dictfilename = global_dir + '/dicts/TestCompSep-JC-Sept2021.dict'

# Read dictionary chosen
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
d['nside']=256


# QUBIC Coverage

In [ ]:
rc('figure', figsize=(16, 4))

covmap = hp.ud_grade(hp.read_map('QUBIC-Typical-Coverage.fits'), d['nside'])

fskytot=np.sum(covmap/np.max(covmap)/len(covmap))*100
suptitle('Cut-Sky (from QUBIC)', fontsize=30, y=1.2)
hp.mollview(covmap, title='QUBIC Cov Map: fsky={0:4.2f}%'.format(fskytot), sub=(1,2,1))

thr = 0.1
mymask = (covmap > (np.max(covmap)*thr)).astype(int)
pixok = mymask > 0
fsky=np.sum(covmap[pixok]/np.max(covmap)/len(covmap))*100
hp.mollview(mymask, title='Mask with thr={0:}: fsky={1:4.2f}'.format(thr, fsky), sub=(1,2,2))


In [ ]:
##### Function to get FastSiumlator maps
def get_map_band(d, band, qubic_sky = None, duration=3., nsnew = None, spatial_noise=False, 
                 nbands=1, nf_in=None, coverage=None, seed=None):
    center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
    d['nf_recon'] = nbands
    if nf_in is not None:
        d['nf_sub'] = nf_in
    print(d['nf_sub'])

    # Frequency band
    dA = d.copy()
    dA['filter_nu'] = band * 1e9

    if seed is None:
        seed = np.random.randint(100000)
    sky_config = {'cmb': seed, 'Dust':'d0'}

    if qubic_sky is None:
        Qubic_sky_A = qss.Qubic_sky(sky_config, dA)
    else:
        Qubic_sky_A = qubic_sky

    ##### Getting FastSimulator output maps
    maps_qubicA, maps_noiselessA, maps_noiseA, coverageA = Qubic_sky_A.get_partial_sky_maps_withnoise(spatial_noise=spatial_noise,
                                                                                 Nyears=duration, coverage=coverage)
    pixok = coverageA > 0
    maps_qubicA[:,~pixok,:] = hp.UNSEEN
    maps_noiselessA[:,~pixok,:] = hp.UNSEEN
    maps_noiseA[:,~pixok,:] = hp.UNSEEN
    
    ##### Corresponding frequencies
    Nfreq_edges, nus_edge, nus, deltas, Delta, Nbbands = qubic.compute_freq(dA['filter_nu'],
                                                                        nbands,
                                                                        dA['filter_relative_bandwidth'])
    print(Nfreq_edges, nus_edge, nus, deltas, Delta, Nbbands)

    if nsnew is not None:
        print('Changinf to nside={}'.format(nsnew))
        sh = np.shape(maps_qubicA)
        new_maps = np.zeros((sh[0], 12*nsnew**2, 3))
        for i in range(sh[0]):
            for j in range(3):
                new_maps[i,:,j] = hp.ud_grade(maps_noiseA[i,:,j], nsnew)
        maps_noiseA = new_maps
        coverageA = hp.ud_grade(coverageA, nsnew, power=-2)
    
    return maps_qubicA, maps_noiselessA, maps_noiseA, coverageA, Qubic_sky_A, nus

In [ ]:
# Function to get QUBIC maps
def get_qss_maps(d, duration=3, freq=150, nsub=3, seed=42, spatial_noise=True, coverage=None):
    if seed is None:
        seed = np.random.randint(100000)
    freq = np.atleast_1d(freq)
    nsub = np.atleast_1d(nsub)
    nmaps = np.sum(nsub)
    maps_qubic = np.zeros((nmaps, 3, 12*d['nside']**2))
    maps_noiseless = np.zeros((nmaps, 3, 12*d['nside']**2))
    maps_noise = np.zeros((nmaps, 3, 12*d['nside']**2))
    nus = np.zeros(nmaps)
    istart = 0
    for i in range(len(freq)):
        print('\nSimulating {} GHz band with {} sub-bands'.format(freq[i], nsub[i]))
        m, m_noiseless, m_noise, cov, qs, nnn = get_map_band(d,freq[i], 
                                                                    duration=duration, 
                                                                    nbands=nsub[i], 
                                                                    spatial_noise=spatial_noise, 
                                                                    coverage=covmap, 
                                                                    seed=seed)
        maps_qubic[istart:istart+nsub[i],:,:] = np.transpose(m, axes=(0,2,1))
        maps_noiseless[istart:istart+nsub[i],:,:] = np.transpose(m_noiseless, axes=(0,2,1))
        maps_noise[istart:istart+nsub[i],:,:] = np.transpose(m_noise, axes=(0,2,1))
        nus[istart:istart+nsub[i]] = nnn
        istart += nsub[i]

    weights=1
    bpconv = utils.bandpass_unit_conversion(150 * u.GHz, weights,u.uK_CMB)
    okpix = cov!=0
    sky_emissions_dust = qs.sky.components[0].get_emission(150 * u.GHz) * bpconv
    maps_dust = np.array([emission.value for emission in sky_emissions_dust])
    maps_dust[:,~okpix] = hp.UNSEEN
    
    sky_emissions_cmb = qs.sky.components[1].get_emission(150 * u.GHz) * bpconv
    maps_cmb = np.array([emission.value for emission in sky_emissions_cmb])
    maps_cmb[:,~okpix] = hp.UNSEEN
    maps_truth_infres = [maps_dust, maps_cmb]

    return maps_qubic, maps_noiseless, maps_noise, nus, maps_truth_infres



In [ ]:
spatial_noise=False
nsub = [2, 2]
band = [150, 220]
duration = 30000000
seed = 42

#### Figure with error for each band...
maps_qubic, maps_noiseless, maps_noise, nus, maps_truth = get_qss_maps(d, duration=duration, freq=band, nsub=nsub, 
                                                spatial_noise=spatial_noise, coverage=covmap, seed=seed)


In [ ]:
csjc.display_maps(maps_qubic, rot=center, freqs=nus/1e9, figsize=(16,20))
tight_layout()

In [ ]:
csjc.display_maps(maps_truth[0], rot=center, mytitle='Input Dust')
csjc.display_maps(maps_truth[1], rot=center, mytitle='Input CMB')

In [ ]:
reload(fgb)
reload(csjc)
freqs = nus / 1e9
synthbeam_peak150_fwhm = 0.39268176  # in degrees
myFWHM = synthbeam_peak150_fwhm * 150. / freqs # Qubic beams
print(myFWHM)
ref_fwhm = np.max(myFWHM)*0+0.5

result_pix_qubic = csjc.apply_fgb(maps_qubic, freqs, myFWHM, verbose=True, resol_correction=True, 
                                  ref_fwhm=ref_fwhm,
                                  plot_separated=True, 
                                  center=center,
                                  truth=maps_truth, plot_residuals=True)

In [ ]:
result_alm_qubic = csjc.apply_fgb(maps_qubic, freqs, myFWHM, verbose=True, resol_correction=True, 
                                  ref_fwhm=ref_fwhm, alm_space=True,
                                  plot_separated=True, 
                                  center=center,
                                  truth=maps_truth, plot_residuals=True)

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:,.5g}'.format

data = [np.append(result_pix_qubic[0], np.ravel([result_pix_qubic[4], result_pix_qubic[3]])),
       np.append(result_alm_qubic[0], np.ravel([result_alm_qubic[4], result_alm_qubic[3]]))]
lines = ['Pixel Based - QSS Maps - Ref FWHM=0.5',
        'Alm Based - QSS Maps - Ref FWHM=0.5']
names = ['beta','CMB I RMS', 'CMB Q RMS', 'CMB U RMS', 'Dust I RMS', 'Dust Q RMS', 'Dust U RMS']
pd.DataFrame(data,lines, names)

Why does it find non zero residuals wheen beta is found to be 1.54 (nsub=1, nsub=1] ????

OK for some convolution residuals,..but apart from this it should be essentially perfect ?

In [ ]:
fgb?